## Part 1: Pulse Rate Algorithm

### Contents
Fill out this notebook as part of your final project submission.

**You will have to complete both the Code and Project Write-up sections.**
- The [Code](#Code) is where you will write a **pulse rate algorithm** and already includes the starter code.
   - Imports - These are the imports needed for Part 1 of the final project. 
     - [glob](https://docs.python.org/3/library/glob.html)
     - [numpy](https://numpy.org/)
     - [scipy](https://www.scipy.org/)
- The [Project Write-up](#Project-Write-up) to describe why you wrote the algorithm for the specific case.


### Dataset
You will be using the **Troika**[1] dataset to build your algorithm. Find the dataset under `datasets/troika/training_data`. The `README` in that folder will tell you how to interpret the data. The starter code contains a function to help load these files.

1. Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

-----

### Code

In [3]:
import glob

import numpy as np
import scipy as sp
import scipy.io
import scipy.signal



fs = 125  
minBPM = 40  
maxBPM = 240  
window_length = 8 * fs  
window_shift = 2 * fs  


def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]


def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)

def bandpass_filter(signal, fs=125, order=3, window=[40/60, 240/60]):
    """filter the signal between 40 and 240 BPM

    Args:
        signal: input signal
        fs: sampling freq

    Returns:
        new signal  withing the range of the desired frequencies
    """
    
    b, a = scipy.signal.butter(order, window, btype='bandpass', fs=fs)
    return scipy.signal.filtfilt(b, a, signal)


def fft(signal, fs = 125):
    """
    computes the forier transform and ft frequency of the passed signal
    
    Args:
        signal: input signal
        fs: Hz of input signal

    Returns:
        fft: fourier transform
        freqs: fourier transformed frequencies
    """
    n_samples = len(signal)  # No zero padding
    freqs = np.fft.rfftfreq(n_samples, 1/fs)
    fft = np.abs(np.fft.rfft(signal, n_samples))
    return freqs, fft

def calculate_magnitude(x, y, z):
    """
    Calculate the magnitude for the signal

    Args:
        x: x-axis
        y: y-axis
        z: z-axis

    Returns:
        Signal magnitude value
    """
    return np.sqrt(x**2 + y**2 + z**2) 


def RunPulseRateAlgorithm(data_fl, ref_fl):
    """Handler function for computing the pulse rate

    Args:
        data_fl (.mat file ): ppg and acc data
        ref_fl (.mat file): ground truth data

    Returns:
        errors (np_array): difference between ground truth and predictions
        confidence (np_array): confidence values for heart rate predictions
    """
    # load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)

    # bandpass filter the signals
    ppg = bandpass_filter(ppg, fs)
    accx = bandpass_filter(accx, fs)
    accy = bandpass_filter(accy, fs)
    accz = bandpass_filter(accz, fs)
    
    # Consider only magnitude of acceleration
    acc = calculate_magnitude(accx, accy, accz)
    
    # loading the reference file
    ground_truth = sp.io.loadmat(ref_fl)['BPM0'].reshape(-1)
   
    bpm_pred, confidence = [], []

    # Compute both bpm prediction and confidence value    
    for i in range(0, len(ppg) - window_length + 1, window_shift):
        ppg_window = ppg[i:i+window_length]

        # aggregate accelerometer data into single signal to get the acc window
        ppg_window = ppg[i:i+window_length]
        acc_window = acc[i:i+window_length]
        
        # Perform Fourier transform
        ppg_freqs, ppg_fft = fft(ppg_window)
        acc_freqs, acc_fft = fft(acc_window)
        
        # Remove unwanted frequencies 

        # Filter data
        ppg_fft[ppg_freqs <= 40/60.0] = 0.0
        ppg_fft[ppg_freqs >= 240/60.0] = 0.0
        acc_fft[acc_freqs <= 40/60.0] = 0.0
        acc_fft[acc_freqs >= 240/60.0] = 0.0
    
        # Find peaks
        ppg_peaks = sp.signal.find_peaks(ppg_fft, height=2000)[0]
        acc_peaks = sp.signal.find_peaks(acc_fft, height=None)[0]
        
        # Pick frequency with largerst FFT coefficient
        ppg_max = ppg_freqs[np.argmax(ppg_fft)]
        acc_max = acc_freqs[np.argmax(acc_fft)]
        
        if(acc_max != ppg_max):
            est_bpm = ppg_max * 60
                
        # Append largest BPM and calculated confidence computed         
        bpm_pred.append(est_bpm)
        
        # Calculating Confidence
        conf_wind = (ppg_freqs > ppg_max - 1) & (ppg_freqs < ppg_max + 1)  
        conf = np.sum(ppg_fft[conf_wind]) / np.sum(ppg_fft)

        confidence.append(conf)
        
        
    # Calculate errors
    errors = np.abs(np.subtract(bpm_pred, ground_truth))
        
    # Return per-estimate mean absolute error and confidence as a 2-tuple of numpy arrays.
    return np.array(errors), np.array(confidence)


In [4]:
Evaluate()

20.392351067058289

-----
### Project Write-up

Answer the following prompts to demonstrate understanding of the algorithm you wrote for this specific context.

> - **Code Description** - Include details so someone unfamiliar with your project will know how to run your code and use your algorithm. 
> - **Data Description** - Describe the dataset that was used to train and test the algorithm. Include its short-comings and what data would be required to build a more complete dataset.
> - **Algorithhm Description** will include the following:
>   - how the algorithm works
>   - the specific aspects of the physiology that it takes advantage of
>   - a describtion of the algorithm outputs
>   - caveats on algorithm outputs 
>   - common failure modes
> - **Algorithm Performance** - Detail how performance was computed (eg. using cross-validation or train-test split) and what metrics were optimized for. Include error metrics that would be relevant to users of your algorithm. Caveat your performance numbers by acknowledging how generalizable they may or may not be on different datasets.

**Code Description**: The code can be excuted by calling the Evaluate() function, which will load the data, compute, and return the aggregated error metric.

**Data Description**: The data provided is for 3-dimentional accelerometer sensor (x,y,z)-axis as well as a PPG sensor signal.

**Algorithhm Description**: After loading the data, it is then passed to a bandpass filter to eliminate unnecessary range based on the required cut-off frequencies. The signal above the upper cut-off and lower than the lowest cut-off are eliminated. The accelerometer magnitude is then computed by finding the absolute sum of square of the 3-axises. Then both BPM and confidence values are computed based on a 8s window length. Once the heart rate s estimatd, the error is computed and returned as a final output.

**Algorithm Performance**: The Mean Absolute Error at 90% was the main metric for evaluating the performance between the estimated and reference ECG signal at 90% availability.

-----
### Next Steps
You will now go to **Test Your Algorithm** to apply a unit test to confirm that your algorithm met the success criteria. 